#모델 추론 결과값 저장



### 목표 
- Detectron2 Faster RCNN 모델 3가지에 대해서 COCO Val2017 데이터를 이용하여 추론 결과 저장
- R101-DC5
- R101-FPN
- X101-FPN



# 1. 데이터 준비


## 1) 자원 확인

In [ ]:
!nvidia-smi
!nproc

In [ ]:
%mkdir coco
%cd coco
%mkdir images
%cd images


In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

In [ ]:
!unzip val2017.zip
%rm val2017.zip
%cd ../

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

In [ ]:
!unzip annotations_trainval2017.zip
%rm annotations_trainval2017.zip

In [ ]:
%pwd

In [ ]:
dataset_path = '/content/coco/images/val2017/'

## 3-1) 데이터셋 수량 확인

In [ ]:
from glob import glob
img_list = glob(dataset_path + '*.jpg')
print('coco val2017 data count (raw): ' + str(len(img_list)))

## 3-2) 화면 출력을 통해 데이터 이미지의 정상 로드 확인 

In [ ]:
from IPython.display import Image, clear_output
Image(filename=img_list[3], width=300)

# 2. 모델 추론
- Detectron2 및 필요한 패키지 설치    
reference : https://github.com/facebookresearch/detectron2

In [ ]:
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

- torch, detectron2 버전 체크

In [ ]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetMapper, build_detection_test_loader, build_detection_train_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from google.colab.patches import cv2_imshow
import numpy as np
import os, json, cv2, random
import logging
import datetime
import time

## 1) detectron2 데이터셋

### 1-1) detectron2 데이터셋 등록

In [ ]:
register_coco_instances("coco_2017", {}, "/content/coco/annotations/instances_val2017.json", dataset_path)  

### 1-2) 등록된 데이터셋 체크

In [ ]:
dataset_dicts = DatasetCatalog.get("coco_2017")
coco_2017_metadata = MetadataCatalog.get("coco_2017")

for d in random.sample(dataset_dicts, 3):
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=coco_2017_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

## 2) 모델 세팅

In [ ]:
cfg0 = get_cfg()
cfg0.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"))
cfg0.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml")
cfg1 = get_cfg()
cfg1.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg1.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg2 = get_cfg()
cfg2.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg2.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor0 = DefaultPredictor(cfg0)
predictor1 = DefaultPredictor(cfg1)
predictor2 = DefaultPredictor(cfg2)

## 3) 추론 결과 시각화

In [ ]:
test_metadata = MetadataCatalog.get("coco_2017")
test_dataset_dicts = DatasetCatalog.get("coco_2017")
for d in random.sample(test_dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor0(im)  
    v = Visualizer(im[:, :, ::-1], metadata=test_metadata, scale=0.4)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

## 4) cocoEvaluator를 이용하여 AP, AR 값 평가 

In [ ]:
def cocoEval(predictor, cfg):
  evaluator = COCOEvaluator("coco_2017", output_dir=cfg.OUTPUT_DIR)
  val_loader = build_detection_test_loader(cfg, "coco_2017")
  print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
#cocoEval(predictor0, cfg0)

In [ ]:
#cocoEval(predictor1, cfg1)

In [ ]:
#cocoEval(predictor2, cfg2)

## 5) 추론 결과 저장

PredictionString: 결과물 리스트. 각 Bounding box 당 label, score, x1, y1, x2, y2 (ex. "62 0.99 597.3 518.4 954.3 1021.3")    
file_name: 이미지 파일 이름. (ex. 000000000139.jpg)    
width: 이미지 너비. (ex. 640)    
height: 이미지 높이. (ex. 426)    
image_id: dataset에 등록된 id. (ex. 139)    

In [ ]:
import csv
from tqdm import tqdm

# 추론 결과물을 위에서 정의한 형식으로 csv 파일 생성
def to_csvfile(predictor, dataset_dicts, file_path ='/content/results.csv'):
  file_name = file_path.split('/')[-1]
  dir_path = file_path[:len(file_path) - len(file_name)]
  if not os.path.exists(dir_path): 
    file_path = '/content/' + file_name

  with open(file_path, 'w') as submission_csv:
    csv_writer = csv.DictWriter(submission_csv, fieldnames=['PredictionString', 'file_name', 'width', 'height','image_id'])
    csv_writer.writeheader()

    for d in tqdm(dataset_dicts):
      image_path = d["file_name"]
      im = cv2.imread(image_path)
      outputs = predictor(im)
      PredictionString = ''
      classes = outputs["instances"].to("cpu").pred_classes.numpy()
      bboxes = outputs["instances"].to("cpu").pred_boxes.tensor.numpy()
      scores = outputs["instances"].to("cpu").scores.numpy()
      for c_idx in range(len(classes)):
        PredictionString += f'{classes[c_idx]} {scores[c_idx]} {bboxes[c_idx][0]} {bboxes[c_idx][1]} {bboxes[c_idx][2]} {bboxes[c_idx][3]} '
      csv_writer.writerow({'PredictionString': PredictionString, 'file_name': image_path.split('/')[-1], 'width': d["width"],'height': d["height"],'image_id': d["image_id"]})

In [ ]:
to_csvfile(predictor0, test_dataset_dicts, '/content/results0.csv')

In [ ]:
to_csvfile(predictor1, test_dataset_dicts, '/content/results1.csv')

In [ ]:
to_csvfile(predictor2, test_dataset_dicts, '/content/results2.csv')

In [ ]:
!pip install ensemble-boxes

In [ ]:
import os, json, time
import pandas as pd
import numpy as np
from ensemble_boxes import *
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

id_matching = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32,
               33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
               60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]

In [ ]:
def load_csv(path):
    if os.path.isfile(path):  # path 파일이 있다면
        with open(path, 'r', encoding='UTF-8') as csvf:
            data = pd.read_csv(csvf)
        return data
    else:
        print('path is wrong')

In [ ]:
def convert_format_step2(labels_list, scores_list, boxes_list, data, index, h, w):
    pstring = data['PredictionString'][index]
    if type(pstring) is float and np.isnan(pstring):  # 해당 이미지에 대한 결과가 없다면 리턴
        return

    n_pstring = np.array(pstring.split(' '))
    n_pstring = np.delete(n_pstring, len(n_pstring) - 1)

    for idx in range(len(n_pstring) // 6):
        idx6 = idx * 6
        label = float(id_matching[int(n_pstring[idx6])])
        score = float(n_pstring[idx6 + 1])
        x1 = float(n_pstring[idx6 + 2]) / w
        y1 = float(n_pstring[idx6 + 3]) / h
        x2 = float(n_pstring[idx6 + 4]) / w
        y2 = float(n_pstring[idx6 + 5]) / h

        labels_list.append(label)
        scores_list.append(score)
        boxes_list.append([x1, y1, x2, y2])

In [ ]:
def convert_format_step1(labels_lists, scores_lists, boxes_lists, weights, data, row_idx, model_idx, h, w, weights_size):
    labels_list = []
    scores_list = []
    boxes_list = []
    convert_format_step2(labels_list, scores_list, boxes_list, data, row_idx, h, w)
    if len(labels_list) != 0:
        weights.append(weights_size[model_idx])
        labels_lists.append(labels_list)
        scores_lists.append(scores_list)
        boxes_lists.append(boxes_list)


In [ ]:
def do_ensemble(save_path, datas, weights_size):
    with open(save_path, 'wt', encoding='UTF-8') as coco:
        ann = []
        start0 = time.time()
        for row_idx in tqdm(range(len(datas[0]))):
            image_id = datas[0]['image_id'][row_idx]
            w = datas[0]['width'][row_idx]
            h = datas[0]['height'][row_idx]

            labels_lists = []
            scores_lists = []
            boxes_lists = []
            weights = []

            for model_idx in range(len(datas)):
                convert_format_step1(labels_lists, scores_lists, boxes_lists, weights, datas[model_idx], row_idx,
                                     model_idx, h, w, weights_size)

            if len(labels_lists) == 0:
                continue
            else:
                # boxes, scores, labels = nms(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.5) # 0.439
                # boxes, scores, labels = soft_nms(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.5, sigma=0.1, thresh=0.0001) # 0.440
                # boxes, scores, labels = non_maximum_weighted(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.5, skip_box_thr=0.0001) # 0.447
                # boxes, scores, labels = weighted_boxes_fusion(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.5, skip_box_thr=0.0001) # 0.454
                # boxes, scores, labels = weighted_boxes_fusion(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.6, skip_box_thr=0.001) # 0.459
                boxes, scores, labels = weighted_boxes_fusion(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.7, skip_box_thr=0.001)  # 0.460
                # boxes, scores, labels = weighted_boxes_fusion(boxes_lists, scores_lists, labels_lists, weights=weights, iou_thr=0.8, skip_box_thr=0.001) # 0.459

                for c_idx in range(len(labels)):
                    ann += [{'image_id': int(image_id),
                             'category_id': int(labels[c_idx]),
                             'bbox': [boxes[c_idx][0] * w,
                                      boxes[c_idx][1] * h,
                                      (boxes[c_idx][2] - boxes[c_idx][0]) * w,
                                      (boxes[c_idx][3] - boxes[c_idx][1]) * h],
                             'score': float(scores[c_idx]), }]

        json.dump(ann, coco)
    print('{} images, duration time : {} [sec]'.format(len(datas[0]), time.time() - start0))


In [ ]:
# 1. file load
path0 = '/content/results0.csv'  # 40.631
path1 = '/content/results1.csv'  # 42.036
path2 = '/content/results2.csv'  # 43.047
data0 = load_csv(path0)
data1 = load_csv(path1)
data2 = load_csv(path2)
datas = []
datas.append(data0)
datas.append(data1)
datas.append(data2)

In [ ]:
# 2. format convert & ensemble
weights_size = [1, 2, 3]
save_path = '/content/ensemble.json'
do_ensemble(save_path, datas, weights_size)

In [ ]:
# 3. evaluate COCO AP
coco_gt = COCO('/content/coco/annotations/instances_val2017.json')
image_ids = sorted(coco_gt.getImgIds())
coco_dt = coco_gt.loadRes(save_path)
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.params.imgIds = image_ids
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

In [ ]:
#import os
#os._exit(00)